# Notes Report
This report will answer:
1. What do the notes contain? -- How do they differ structurally? Semantically? 
2. Statistical info about the note category. Token count?
3. Statistical comparison between categories

In [1]:
import pandas as pd
rad_path = f'/data/datasets/mimiciv_notes/physionet.org/files/mimic-iv-note/2.2/note/radiology.csv'
dis_path = f'/data/datasets/mimiciv_notes/physionet.org/files/mimic-iv-note/2.2/note/discharge.csv'
rad_cohort_path = f'/home/ugrads/a/aa_ron_su/BoxHED2.0_Fuse/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT_rad.csv'
dis_cohort_path = f'/home/ugrads/a/aa_ron_su/BoxHED2.0_Fuse/JSS_SUBMISSION_NEW/data/till_end_mimic_iv_extra_features_train_NOTE_TARGET1_FT.csv'
rad_c_df = pd.read_csv(rad_cohort_path)
dis_c_df = pd.read_csv(dis_cohort_path)

## 1. What do the notes contain?

The discharge table contains discharge summaries for hospitalizations. Discharge summaries are long form narratives which describe the reason for a patient’s admission to the hospital, their hospital course, and any relevant discharge instructions. 

The radiology table contains free-text radiology reports associated with radiography imaging. Radiology reports cover a variety of imaging modalities: x-ray, computed tomography, magnetic resonance imaging, ultrasound, and so on. Free-text radiology reports are semi-structured and usually follow a consistent template for a given imaging protocol

In [2]:
import numpy as np
import pprint

np.random.seed(1)
n = rad_c_df.shape[0]
idxs = [np.random.randint(n) for _ in range(3)]

print(rad_c_df.iloc[idxs[0]].text)
print( '-' * 40)
'''
HISTORY
FINDINGS
'''

print(rad_c_df.iloc[idxs[1]].text)
print( '-' * 40)
'''
INDICATION
COMPARISON
TECHNIQUE
FINDINGS
IMPRESSION
'''

print(rad_c_df.iloc[idxs[2]].text)
print( '-' * 40)
'''
EXAMINATION
INDICATION
DOSE
COMPARISON
FINDINGS
Head_CTA
IMPRESSION
'''

HISTORY:  Status post CABG.

FINDINGS:  In comparison with earlier study of this date, the mediastinal
tubes and left chest tube have been removed.  No evidence of pneumothorax. 
Opacification at the left base persists consistent with pleural effusion and
atelectasis.  The right lung is essentially clear.

----------------------------------------
INDICATION:  ___ man with intermittent bladder obstruction and new
renal failure, ? hydronephrosis, other causes of renal failure.

COMPARISON:  Renal ultrasound of ___ and bladder ultrasound of
___.

TECHNIQUE:  Gray-scale ultrasound images of the kidneys are obtained.

FINDINGS:

Right kidney measures 9.52 cm.
Left kidney measures 10.8 cm.

Left and right kidneys do not show any hydronephrosis, masses, or stones.

Enlarged prostate gland extends into the base of the bladder. Foley baloon is
fully expanded and the bladder is decompressed.
 
IMPRESSION:  No hydronephrosis.

----------------------------------------
EXAMINATION:  CTA HEAD WANDW/

'\nEXAMINATION\nINDICATION\nDOSE\nCOMPARISON\nFINDINGS\nHead_CTA\nIMPRESSION\n'

Radiology is structured with headings in all caps. These vary in type, but generally there is some form of imaging performed followed by findings, impression, comparison, etc.

In [3]:
import numpy as np

np.random.seed(1)
n = dis_c_df.shape[0]
idxs = [np.random.randint(n) for _ in range(3)]

print(dis_c_df.iloc[idxs[0]].text)
print( '-' * 40)
'''
personal info... (hidden data for privacy)
  Service: Neurosurgery
  Chief Complaint: Fall
  Major Surgical or Invasive Procedure: None
  Name:  ___.           Unit No:   ___
  Admission Date:  ___              Discharge Date:   ___
  Date of Birth:  ___             Sex:   M
  ...
Service: NEUROSURGERY
Chief Complaint:
Major Surgical or Invasive Procedure:
History of Present Illness:
Past Medical History:
Social History:
Family History:
Physical Exam: BP:101 /60         HR:49    R12 CMV 100% FiO2 ... 
  Gen: WD/WN, comfortable, NAD.
  HEENT: Posterior head lac
  Neck: Trauma Collar
  Lungs: CTA bilaterally.
  Cardiac: RRR. S1/S2.
  Abd: Soft, NT, BS+
  Extrem: Warm and well-perfused.
  Neuro:
  Mental status: Intubated, unresponsive
  Cranial Nerves:
  I: Not tested
  II: Pupils right 5mm NR, Left 3 NR
  Positive Corneal on left, no corneal R

  Motor: Weak flex with RUE
  Extension W LUE
  NO responce to Nox BLE

On Discharge:
Expired
 
Pertinent Results:
CT Head:
IMPRESSION: 
Brief Hospital Course:
Medications on Admission:
Discharge Medications: 
Discharge Disposition:
Expired
 
Discharge Diagnosis:
Subdural Hematoma
Respiratory failure
Coma
 
Discharge Condition:
Expired

 
Discharge Instructions:
Expired
 
Followup Instructions:
___

----------------------------------------
 
Name:  ___                    Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
Sulfa (Sulfonamide Antibiotics) / Adhesive Tape
 
Attending: ___.
 
Chief Complaint:
ascitic leakage from surgical site
 
Major Surgical or Invasive Procedure:
diagnostic and therapeutic paracentesis
suture of surgical wound

 
History of Present Illness:
___ presented 8 days ago to OSH with abdominal distention, 
thought to be ___ acute cholecystitis. On insertion of 
laparoscope in OR, gallbladder appeared normal so surgery was 
aborted. Shrunk, fibrotic liver was noted and 4.5 L of ascitic 
fluid was removed during surgery. Hospitalization was also 
complicated by pneumonia, for which levaquin was prescribed (on 
day ___. Pt was planned to f/u at liver clinic. At home this 
am, patient noted ascites draining through one of the 
laparoscope port sites. No purulence/erythema, f/c, n/v/abd 
pain. Admitted for expedited liver workup. Workup negative for 
liver so far, no autoimmune studies sent yet. INR 1.7, Alb 3.3, 
labs otherwise wnl. Initial VS 98.0 83 116/87 16 96RA.  
.  
On the floor, patient had diagnostic tap and only 0.5L could be 
removed therapeutically ___ patient discomfort.  
.  
Review of sytems:  
(+) Per HPI, also notes diarrhea for past ___ weeks, no blood in 
stool; no antibiotics prior to onset. ?associated with lactose 
intolerance. Was prescribed immodium, which has resolved 
diarrhea for the most part. +weight gain and increasing 
abdominal girth.  
(-) Denies fever, chills, night sweats. Denies headache, sinus 
tenderness, rhinorrhea or congestion. Denied cough, shortness of 
breath (never any symptoms of pneumonia). Denied chest pain or 
tightness, palpitations. Denied nausea, vomiting, diarrhea, 
constipation or abdominal pain. No dysuria. Denied arthralgias 
or myalgias. 

 
Past Medical History:
1. Hypertension  
2. Hyperlipidemia  
3. Type II diabetes (___)  
4. OSA on CPAP at home  
5. Osteoporosis  
6. Barrett's esophagitis (EGD ___ yrs ago)  
7. LLL pneumonia  
8. Cirrhosis of the liver  
9. COPD (no hx smoking)  
10. s/p hysterectomy  
11. s/p laparascopy  
12. s/p cataracts surgery  
___. diverticulosis (on colonoscopy in ___ 

 
Social History:
___
Family History:
Son and two brothers w/ hemachromatosis (two brothers passed 
away from cardiac cx). Esophageal ca in sister.
 
Physical Exam:
General: Alert, oriented, no acute distress; +spider angiomas  
HEENT: Sclera anicteric, MMM, oropharynx clear  
Neck: supple, JVP not elevated, no LAD  
Lungs: Clear to auscultation bilaterally, no wheezes, rales, 
ronchi  
CV: Regular rate and rhythm, normal S1 + S2, no murmurs, rubs, 
gallops  
Abdomen: healing laparoscopy wound in umbilicus C/D/I; midline 
laparoscopy wound w/ serous fluid draining, no pus or erythema; 
abdomen is soft, non-tender, distended w/ ascites, bowel sounds 
present, no rebound tenderness or guarding; +hepatosplenomegaly  

GU: no foley  
Ext: warm, well perfused, 2+ pulses, no clubbing, cyanosis; 
1+pitting edema b/l  
Neuro: No asterixis, AAOx3, ___ mm strength in all 4 
extremities; no deficits in fine touch sensation or 
proprioception in all four extremities.  

 
Pertinent Results:
From OSH: HBSAg-negative, HbSAb-negative, HBcAb-negative, 
anti-HAv negative, anti-HCV negative, alpha-1-anti-trypsin 
levels low at 34, AFP- 3.1, ___, Ferritin 242  

___ 01:50PM   GLUCOSE-112* UREA N-18 CREAT-1.0 SODIUM-137 
POTASSIUM-4.4 CHLORIDE-105 TOTAL CO2-26 ANION GAP-10
___ 01:50PM   ALT(SGPT)-35 AST(SGOT)-51* ALK PHOS-154* TOT 
BILI-0.7
___ 01:50PM   LIPASE-24
___ 01:50PM   ALBUMIN-3.3*
___ 01:50PM   WBC-10.8 RBC-4.14* HGB-14.2 HCT-42.3 MCV-102* 
MCH-34.3* MCHC-33.5 RDW-13.2
___ 01:50PM   NEUTS-69.1 ___ MONOS-7.6 EOS-0.9 
BASOS-0.5
___ 01:50PM   PLT COUNT-229
___ 02:55PM   ___ PTT-26.7 ___

___ 05:48PM ASCITES  WBC-158* RBC-46* POLYS-8* LYMPHS-11* 
MONOS-20* MESOTHELI-6* MACROPHAG-55*
___ 05:48PM ASCITES  TOT PROT-0.3 ALBUMIN-<1.0

___-
HBsAg, HBsAb, HBcAb, HCV: negative
AMA: negative
Anti-Smooth muscle Ab: **POSITIVE; titer 1:20
___: POSITIVE, 1:40, speckled pattern
IgG: 1576
HHE mutation analysis: PENDING

calTIBC ___
Ferritin 201
TRF 143
Iron 69

Alpha-1 antitrypsin level: 31
Alpha-1 antitrypsin phenotype:
Test Name                  In Range   Out of Range     Reference 
Range
---------                  --------   ------------     
---------------
Alpha-1-Antitrypsin Phenotype, Serum
  Alpha-1-Antitrypsin Pheno  ZZ                        band
 
Expected Values
---------------
Most normal individuals have the M phenotype (M, M-1, or M-2). 
Over 99% of
M phenotypes are genotypically MM. In the absence of family 
studies, the 
phenotype (M) and quantitative level can be used to infer to 
genotype (MM). The most common alleles associated with a 
quantitative deficiency are Z and S. The reports for the rare 
alleles will indicate whether or not they havebeen associated 
with reduced quantitative levels. 
 
  Alpha-1-Antitrypsin, S                  31  L        100-190 
mg/dL

MICROBIOLOGY:
Peritoneal Culture (___): No Growth (preliminary)

IMAGING:
Abd U/S w/ dopplers (___)
1. Cirrhotic-appearing liver. Sensitivity for detection of focal 
liver 
lesions is limited by ultrasound. Contrast-enhanced CT or MR is 
recommended 
for further evaluation. 
2. Ascites. 
3. Patent portal vein. 

 
Brief Hospital Course:
___ w/ PMH DM, HTN, HL, Barrett's esophagitis, cirrhosis, who 
has strong family hx of hemachromatosis and presents after 
ascitic fluid drainage from laparoscopy site.  
 
#. Ascitic fluid leakage: Leakage was serous and nonpurulent and 
patient remained afebrile with no abdominal pain throughout 
hospitalization. The most serious concern was for secondary 
peritonitis, so diagnostic tap was performed w/ no evidence of 
infection.  Therapeutic tap yielded only 500cc of fluid and was 
terminated ___ patient discomfort.  General surgery was 
consulted and placed horizontal mattress sutures, which can be 
removed by Ms. ___ PCP no earlier than ___. 
.  
#. Cirrhosis: Etiology of the patient's cirrhosis was unclear on 
admission. She has no history of long-standing ETOH use or risk 
factors for hepatitis infection. Hepatitis panel was negative. 
She does have a strong family history of hemachromatosis, but 
this diagnosis is not strongly favored by iron study results. 
Mutation analyses were still pending at the time of this 
summary. The patient does have a low alpha-1 antitrypsin, and 
mutation analysis suggests that she is ZZ (Z alleles associated 
with quantitative deficiency).  Interestingly, she also carries 
a diagnosis of COPD with no history of personal tobacco use. In 
addition, she is positive for anti-smooth muscle antibody with a 
weakly positive ___, but normal IgG. This might suggest a 
component of autoimmune hepatitis in addition to alpha-1 
antitrypsin deficiency.  
She was continued on lasix 40mg daily, spironolactone 50mg BID.  
She will need an EGD as an outpatient to evaluate for esophageal 
varices in the next year (last ___ yrs ago). She will follow up in 
___ clinic. 
.  
#. Pneumonia: Pneumonia was diagnosed at OSH for which she was 
prescribed a 7 day course of levaquin.  She finished her course 
on ___ and exam on both admission and discharge revealed 
clear lung fields.  She remained afebrile and asymptomatic.
 
Medications on Admission:
1. Norvasc 5mg daily  
2. ASA 81mg daily  
3. Symbicort 2puffs BID  
4. Calcium/Vitamin D  
5. Nexium 40mg daily  
6. Glyburide 1.25mg daily  
7. Xopenex 2puffs Q4-6hrs PRN  
8. Boniva 150mg PO Qmonth  
9. Centrum daily  
10. Zocor 80mg daily  
11. Januvia 100mg daily  
12. Diovan 80mg daily  
13. Aldactone 100mg BID  
14. Levaquin 750mg daily until ___ 

 
Discharge Medications:
1. Amlodipine 5 mg Tablet Sig: One (1) Tablet PO DAILY (Daily).  

2. Aspirin 81 mg Tablet, Chewable Sig: One (1) Tablet, Chewable 
PO DAILY (Daily).  
3. Simvastatin 80 mg Tablet Sig: One (1) Tablet PO once a day.  
4. Valsartan 80 mg Tablet Sig: One (1) Tablet PO once a day.  
5. Symbicort 160-4.5 mcg/Actuation HFA Aerosol Inhaler Sig: Two 
(2) puffs Inhalation twice a day.  
6. Esomeprazole Magnesium 40 mg Capsule, Delayed Release(E.C.) 
Sig: One (1) Capsule, Delayed Release(E.C.) PO once a day.  
7. Calcium Carbonate-Vitamin D3 500 mg(1,250mg) -400 unit Tablet 
Sig: One (1) Tablet PO twice a day.  
8. Glyburide 1.25 mg Tablet Sig: One (1) Tablet PO once a day.  
9. Levalbuterol Tartrate 45 mcg/Actuation HFA Aerosol Inhaler 
Sig: ___  Inhalation every ___ hours as needed for shortness of 
breath or wheezing.  
10. Ibandronate 150 mg Tablet Sig: One (1) Tablet PO once a 
month.  
11. Multivitamin     Tablet Sig: One (1) Tablet PO DAILY 
(Daily).  
12. Sitagliptin 100 mg Tablet Sig: One (1) Tablet PO once a day. 
 
13. Furosemide 40 mg Tablet Sig: One (1) Tablet PO DAILY 
(Daily).
Disp:*30 Tablet(s)* Refills:*2*
14. Spironolactone 50 mg Tablet Sig: One (1) Tablet PO twice a 
day.
Disp:*60 Tablet(s)* Refills:*2*
15. Outpatient Lab Work
Please have a Chem 7 (Na,K,Cl,HCO3,BUN,Creatinine,Glucose) drawn 
2 weeks after discharge with results faxed to Dr. ___ at 
___.

 
Discharge Disposition:
Home
 
Discharge Diagnosis:
1. Dehiscence of surgical wound
2. Cirrhosis with ascites

 
Discharge Condition:
Stable vital signs. Afebrile. 

 
Discharge Instructions:
You were admitted to ___ on ___ with 
drainage from your laparascopic site.  The fluid was from your 
ascites and did not have any infection.  The surgical team put 
in a suture to keep the wound closed. You can have this suture 
removed in 2 weeks by your PCP.

You will need to follow up as an outpatient with your GI doctor 
and with the hepatologists. We have drawn several blood tests to 
attempt to determine the cause of your cirrhosis.  The results 
of these tests will be made available to your doctors and ___ 
___ follow up as listed below.

We have made the following medication changes:
START furosemide (you will need electrolytes checked in 2 weeks 
by your primary care doctor).
CHANGE spironolactone dose to 50mg twice daily

Please return to the ER if you have fevers/chills, abdominal 
pain, shortness of breath, chest pain, bloody stools, worsening 
wound drainage, or any other symptoms concerning to you.
 
Followup Instructions:
___

----------------------------------------
 
Name:  ___                   Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: SURGERY
 
Allergies: 
morphine / Lipitor
 
Attending: ___.
 
Chief Complaint:
porto-venous and mesenteric venous gas; ischemic bowel; sepsis

 
Major Surgical or Invasive Procedure:
___ exploratory laparotomy, extended right colectomy, left 
in discontinuity  
___ Abdominal closure, end ileostomy, long ___ 
___ PEG  

 
History of Present Illness:
___, poor historian, transferred from ___ after CT obtained
today returned with extensive portal and mesenteric venous gas,
pneumatosis and concerns for bowel ischemia. Pt with 3 days of
abdominal cramping with diarrhea x 3 days. Reported to ___ today where initial evaluation was performed. Minimally
resuscitated per records. He reports feeling dehydrated,
weakened, and now with nausea. Foley placed in OSH. Denies any
fevers or dysuria. 

 
Past Medical History:
PMH: CAD, CHF, afib on coum, HL, HTN
PSH: appy, GB, R colectomy with colostomy, takedown 

 
Social History:
___
Family History:
NC
 
Physical Exam:
97.8  86  120/58  18  96%RA
NAD, alert, mumbles some comprehensible words but certainly not 
oriented
Irregularly irregular
No resp distress
Abd soft, obese, NT, ND, PEG in place, ostomy functional, 
incision dry and approximated
Ext wwp x4
 
Pertinent Results:
___ 01:45PM BLOOD WBC-9.4 RBC-5.65 Hgb-18.4* Hct-55.2* 
MCV-98 MCH-32.6* MCHC-33.4 RDW-14.1 Plt ___
___ 01:43AM BLOOD WBC-4.6 RBC-4.34* Hgb-14.3 Hct-42.3 
MCV-97 MCH-33.0* MCHC-33.9 RDW-14.1 Plt ___
___ 02:17AM BLOOD WBC-17.1*# RBC-3.61* Hgb-11.5* Hct-35.8* 
MCV-99* MCH-31.9 MCHC-32.2 RDW-14.0 Plt ___
___ 01:56AM BLOOD WBC-23.0* RBC-3.57* Hgb-11.5* Hct-35.9* 
MCV-101* MCH-32.4* MCHC-32.1 RDW-13.9 Plt ___
___ 06:55AM BLOOD WBC-9.6 RBC-2.93* Hgb-9.6* Hct-29.5* 
MCV-101* MCH-32.9* MCHC-32.7 RDW-13.3 Plt ___

___ 04:55AM BLOOD WBC-11.9* RBC-2.90* Hgb-9.3* Hct-28.9* 
MCV-100* MCH-32.0 MCHC-32.1 RDW-13.4 Plt ___
___ 01:45PM BLOOD ___ PTT-37.1* ___
___ 01:51PM BLOOD ___ PTT-61.5* ___
___ 10:30AM BLOOD ___ PTT-47.3* ___
___ 04:55AM BLOOD ___
___ 01:45PM BLOOD Glucose-150* UreaN-58* Creat-6.2* Na-140 
K-3.4 Cl-98 HCO3-11* AnGap-34*
___ 09:04PM BLOOD Glucose-152* UreaN-52* Creat-4.8* Na-135 
K-3.1* Cl-105 HCO3-16* AnGap-17
___ 01:43AM BLOOD Glucose-118* UreaN-53* Creat-4.9* Na-136 
K-3.7 Cl-103 HCO3-15* AnGap-22*
___ 12:36AM BLOOD Glucose-97 UreaN-61* Creat-4.6* Na-132* 
K-4.1 Cl-101 HCO3-18* AnGap-17
___ 02:13AM BLOOD Glucose-93 UreaN-67* Creat-4.1* Na-128* 
K-3.5 Cl-100 HCO3-18* AnGap-14
___ 02:12AM BLOOD Glucose-83 UreaN-65* Creat-3.8* Na-137 
K-3.3 Cl-107 HCO3-19* AnGap-14
___ 02:03AM BLOOD Glucose-90 UreaN-65* Creat-3.3* Na-141 
K-3.8 Cl-111* HCO3-21* AnGap-13
___ 02:09AM BLOOD Glucose-112* UreaN-74* Creat-3.1* Na-145 
K-4.3 Cl-114* HCO3-22 AnGap-13
___ 02:17AM BLOOD Glucose-106* UreaN-80* Creat-2.8* Na-151* 
K-4.6 Cl-113* HCO3-25 AnGap-18
___ 01:59AM BLOOD Glucose-164* UreaN-84* Creat-2.6* Na-150* 
K-4.2 Cl-113* HCO3-26 AnGap-15
___ 01:56AM BLOOD Glucose-140* UreaN-92* Creat-2.7* Na-146* 
K-4.9 Cl-110* HCO3-26 AnGap-15
___ 02:12AM BLOOD Glucose-103* UreaN-107* Creat-3.2* Na-141 
K-5.3* Cl-101 HCO3-25 AnGap-20
___ 04:19AM BLOOD Glucose-111* UreaN-123* Creat-3.7* 
Na-147* K-4.8 Cl-103 HCO3-28 AnGap-21*
___ 12:22AM BLOOD Glucose-128* UreaN-132* Creat-3.6* 
Na-147* K-4.7 Cl-100 HCO3-30 AnGap-22*
___ 02:07AM BLOOD Glucose-116* UreaN-124* Creat-3.3* Na-142 
K-3.9 Cl-97 HCO3-30 AnGap-19
___ 01:45AM BLOOD Glucose-110* UreaN-109* Creat-2.7* Na-138 
K-4.2 Cl-95* HCO3-29 AnGap-18
___ 06:55AM BLOOD Glucose-112* UreaN-93* Creat-2.2* Na-140 
K-4.3 Cl-100 HCO3-31 AnGap-13
___ 09:30AM BLOOD Glucose-122* UreaN-74* Creat-2.1* Na-146* 
K-4.3 Cl-104 HCO3-28 AnGap-18
___ 05:20AM BLOOD Glucose-125* UreaN-68* Creat-2.0* Na-142 
K-4.3 Cl-103 HCO3-29 AnGap-14
___ 05:45AM BLOOD Glucose-119* UreaN-73* Creat-2.0* Na-145 
K-4.5 Cl-103 HCO3-27 AnGap-20
___ 05:30AM BLOOD Glucose-131* UreaN-81* Creat-2.3* Na-146* 
K-4.3 Cl-106 HCO3-26 AnGap-18
___ 05:30AM BLOOD Glucose-128* UreaN-91* Creat-2.8* Na-143 
K-4.4 Cl-102 HCO3-27 AnGap-18
___ 05:50AM BLOOD Glucose-129* UreaN-97* Creat-3.2* Na-141 
K-4.8 Cl-99 HCO3-26 AnGap-21*
___ 04:55AM BLOOD Glucose-83 UreaN-93* Creat-2.8* Na-143 
K-4.9 Cl-104 HCO3-26 AnGap-18
___ 05:20AM BLOOD Glucose-86 UreaN-75* Creat-2.3* Na-146* 
K-4.2 Cl-107 HCO3-27 AnGap-16
___ 04:55AM BLOOD Glucose-133* UreaN-60* Creat-1.9* Na-145 
K-4.0 Cl-107 HCO3-27 AnGap-15
___ 01:45PM BLOOD ALT-36 AST-28 AlkPhos-83 TotBili-0.5
___ 06:11AM BLOOD ALT-14 AST-37 AlkPhos-33* TotBili-2.1*
___ 10:07PM BLOOD ALT-17 AST-26 AlkPhos-107 TotBili-2.8*
___ 10:00PM BLOOD CK(CPK)-202
___ 05:10AM BLOOD CK(CPK)-236
___ 01:00PM BLOOD CK(CPK)-187
___ 01:45PM BLOOD Lipase-27
___ 01:45PM BLOOD cTropnT-0.05*
___ 01:51PM BLOOD CK-MB-9 cTropnT-<0.01
___ 09:20PM BLOOD CK-MB-8 cTropnT-<0.01
___ 10:00PM BLOOD CK-MB-2 cTropnT-0.10*
___ 05:10AM BLOOD CK-MB-2
___ 01:00PM BLOOD CK-MB-3
___ 01:45PM BLOOD Albumin-4.4 Calcium-9.2 Phos-6.8* Mg-1.9
___ 09:04PM BLOOD Calcium-7.9* Phos-4.3 Mg-2.5
___ 04:55AM BLOOD Calcium-8.9 Phos-3.5 Mg-1.7
___ 09:04PM URINE Color-AMBER Appear-Cloudy Sp ___
___ 09:04PM URINE Blood-LG Nitrite-NEG Protein-100 
Glucose-TR Ketone-NEG Bilirub-NEG Urobiln-NEG pH-5.5 Leuks-LG
___ 09:04PM URINE RBC->182* WBC-159* Bacteri-MOD Yeast-NONE 
Epi-0 TransE-1
___ 09:04PM URINE CastGr-8* CastHy-32*
___ 09:04PM URINE Mucous-OCC
___ 09:04PM URINE Hours-RANDOM Creat-111 Na-19 K-52 Cl-14
___ 04:10PM URINE Color-Yellow Appear-Clear Sp ___
___ 04:10PM URINE Blood-MOD Nitrite-NEG Protein-TR 
Glucose-NEG Ketone-NEG Bilirub-NEG Urobiln-NEG pH-5.5 Leuks-NEG
___ 04:10PM URINE RBC-29* WBC-3 Bacteri-FEW Yeast-NONE 
Epi-<1
___ 04:10PM URINE CastHy-1*
___ 04:10PM URINE Mucous-RARE
___ 02:03AM URINE Hours-RANDOM UreaN-670 Creat-61 Na-21 
K-20 Cl-27
___ 10:13PM URINE Color-Yellow Appear-Hazy Sp ___
___ 10:13PM URINE Blood-MOD Nitrite-NEG Protein-30 
Glucose-NEG Ketone-NEG Bilirub-NEG Urobiln-NEG pH-5.5 Leuks-NEG
___ 10:13PM URINE RBC-7* WBC-4 Bacteri-NONE Yeast-NONE 
Epi-0
___ 10:13PM URINE CastHy-4*
___ 10:13PM URINE Mucous-RARE
___ 10:13PM URINE Hours-RANDOM UreaN-771 Creat-118 Na-10 
K-80 Cl-11
___ 01:30PM URINE Color-Yellow Appear-Clear Sp ___
___ 01:30PM URINE Blood-TR Nitrite-NEG Protein-30 
Glucose-NEG Ketone-NEG Bilirub-NEG Urobiln-NEG pH-6.0 Leuks-NEG
___ 01:30PM URINE Mucous-RARE

The following cultures were all negative:
Blood culture: ___
Urine culture: ___
C.diff: ___

CT ABDOMEN W/O CONTRAST Study Date of ___ 1:14 ___ 
Extensive mesenteric venous gas and portal venous gas is 
concerning for a 
large territory of ischemic bowel.  Ischemic bowel appears 
grossly in the 
distribution of the SMA.  The SMA calcifications are only mild.  
Patency of the SMA cannot be assessed without intravenous 
contrast. 

CT HEAD W/O CONTRAST Study Date of ___ 10:___vidence of acute intracranial process.  Chronic changes 
as described above. MRI would be more sensitive for brainstem 
ischemia. 

CT ABD & PELVIS W/O CONTRAST Study Date of ___ 4:09 ___ 
1.  Status post right colectomy and diverting ileostomy with no 
evidence of fluid collection or abscess formation.   
2.  Small bowel and sigmoid diverticulosis, no evidence of acute 

diverticulitis.   
3.  Left abdominal wall defect containing multiple loops of 
small bowel 
without evidence of bowel strangulation, likely secondary to 
diastasis of the rectus abdominis muscles versus herniation.   

CT CHEST W/O CONTRAST Study Date of ___ 4:23 ___ 
1.  Mild opacification at the lung bases, is most likely 
atelectasis. 
Endobronchial material is either retained secretions or recent 
aspiration.  None of the contrast agent filling the stomach is 
present in the tracheobronchial tree. 
2.  Asbestos-related pleural plaques.  No evidence of 
intrathoracic 
malignancy. 
3.  Mild-to-moderate emphysema. 
4.  Moderate cardiomegaly.  No pulmonary edema. 

CT ABD & PELVIS W/O CONTRAST Study Date of ___ 10:56 ___ 
Postoperative changes from previous ileostomy and right 
hemicolectomy without evidence of obstruction or extraluminal 
fluid collection to suggest abscess. 
Marked bladder distention.  Consideration of Foley catheter 
placement is 
recommended if there is a history of urinary retention. 

MR HEAD W/O CONTRAST Study Date of ___ 2:56 ___ 
No acute infarction.  No evidence for other acute intracranial 
abnormalities. 

 
Brief Hospital Course:
___ INR 2.3, d/c hep gtt, MS slightly improved, nystatin 
for (early) ___  
___ tube feeding resumed, heparin drip  
___ OR for PEG  
___ Foley placed for overflow incontinence  
___ every other staple removed. self d/c'd ___ - 
replaced & bridled  
___ dobhoff replaced, dc'd foley, desat to 90  
___ Increase O2 req day, less responsive on ___ exam, more 
somnolent  
___ ABG, CXR, EKG, serial cardiac enzymes ordered   
Mr. ___ was admitted to the Acute Care Surgery service at 
___ on ___ with 
pneumatosis and portal/mesenteric venous gas, sepsis, and acute 
renal failure. He was promptly taken to the operating room for 
an exploratory laparotomy and right colectomy for ischemia. For 
further details of the procedure, please see the operative 
report. The patient was left in discontinuity and brought to the 
intensive care unit for resuscitation. 

On ___, the pt returned to the operating room for an end 
ileostomy and abdominal closure. He then returned intubated to 
the intensive care unit where his hypotension, arrhythmias (runs 
of vtach, trigeminy), renal failure, and altered mental status 
were managed. Finally, on ___, the patient was extubated. 
On ___, a CT scan was obtained given the patient's 
leukocytosis but did not show any abscesses. On ___, 
Nephrology was consulted regarding the patient's persistent 
renal failure, which they felt was prerenal. The following day, 
the patient was started on 1:1 replacement of his ileostomy 
output with lactated ringers and immodium was started. 

On ___ the patient's diet was advanced to purees after he was 
cleared by speech and swallow. He was started on erythromycin 
for improved motility. The following day, on ___, the patient 
was transferred to the floor. That same day, Speech recommended 
the patient be made NPO for concern of aspiration. He was given 
tube feeds via a dobhoff catheter. On ___, the patient was 
taken back to the operating room for placement of a percutaneous 
gastrostomy tube. 

While on the floor, the patient's foley was removed and then 
replaced for urinary retention. His dobhoff was self d/c'd and 
replaced twice. His mental status waxed and waned although was 
peristently poor. He was bridged back to coumadin with a heparin 
drip. His creatinine and BUN slowly drifted down. 

On ___, the patient's mental status was notably improved. He 
answered some questions appropriately and spoke some 
comprehensible words. He was screened and accepted at a rehab 
facility where he will continue his recovery. 

 
Medications on Admission:
Crestor 20'
Lasix 40'
Lisinopril 40'
KCl 20 meq daily
MVI
Norvasc
Coumadin 5qpm ___ 2.5qpm Th

 
Discharge Medications:
1. Acetaminophen (Liquid) 650 mg PO Q6H:PRN pain 
2. Furosemide 40 mg PO DAILY 
3. LOPERamide 2 mg PO QID:PRN administer for high ostomy output 
4. Nystatin Oral Suspension 5 mL PO QID Duration: 3 Days 
5. Opium Tincture 5 DROP PO Q8H 
6. Rosuvastatin Calcium 20 mg PO DAILY 
___  MD to order daily dose PO DAILY16 

 
Discharge Disposition:
Extended Care
 
Facility:
___
 
Discharge Diagnosis:
ischemic colitis
sepsis
acute renal failure
altered mental status

 
Discharge Condition:
Mental Status: Confused - always.
Level of Consciousness: Alert and interactive.
Activity Status: Out of Bed with assistance to chair or 
wheelchair.

 
Discharge Instructions:
You came to the hospital with abdominal pain and had a CT scan 
which showed insufficient blood flow to your large bowel. You 
were taken to the operating room for exploration and had your 
right colon removed. Two days later, your abdomen was 
re-explored, an ileostomy was made, and your abdomen was closed. 
Due to your altered mental status causing your to be unable to 
eat, you later had a feeding tube placed. Your ileostomy has 
been working and you are tolerating tube feeds. Your mental 
status has been improving and your swallowing ability should be 
evaluated next week.  

Please follow up in ___ clinic.

ACTIVITY:
Don't lift more than ___ lbs for ___ weeks. (This is about the 
weight of a briefcase or a bag of groceries.) This applies to 
lifting children, but they may sit on your lap.
You will need to stay out of bathtubs or swimming pools for a 
time while your incision is healing. Ask your doctor when you 
can resume tub baths or swimming.

HOW YOU MAY FEEL:
You may feel weak or "washed out" for 6 weeks. You might want to 
nap often. Simple tasks may exhaust you.
You might have trouble concentrating or difficulty sleeping. You 
might feel somewhat depressed.
You could have a poor appetite for a while. Food may seem 
unappealing.
All of these feelings and reactions are normal and should go 
away in a short time.  If they do not, tell your surgeon.

YOUR INCISION:
You may gently wash away dried material around your incision.
It is normal to feel a firm ridge along the incision. This will 
go away.
Avoid direct sun exposure to the incision area.
Do not use any ointments on the incision unless you were told 
otherwise.
You may see a small amount of clear or light red fluid staining 
your dressing or clothes. If the staining is severe, please call 
your surgeon.
You may shower. As noted above, ask your doctor when you may 
resume tub baths or swimming.
Ove the next ___ months, your incision will fade and become 
less prominent.

Monitoring Ostomy output/Prevention of Dehydration:
*Keep well hydrated.
*Replace fluid loss from ostomy daily.
*Avoid only drinking plain water. Include Gatorade and/or other 
vitamin drinks to replace fluid.
*Try to maintain ostomy output between 1000mL to 1500mL per day.
*If Ostomy output >1 liter, take 4mg of Imodium, repeat 2mg with 
each episode of loose stool. Do not exceed 16mg/24 hours.

PAIN MANAGEMENT:
You may take tylenol as needed for pain. 
If you experience any of the folloiwng, please contact your 
surgeon:
- sharp pain or any severe pain that lasts several hours
- pain that is getting worse over time
- pain accompanied by fever of more than 101
- a drastic change in nature or quality of your pain

MEDICATIONS:
Take all the medicines you were on before the operation just as 
you did before, unless you have been told differently.
In some cases you will have a prescription for antibiotics or 
other medication.
If you have any questions about what medicine to take or not to 
take, please call your surgeon.

DANGER SIGNS:
Please call your surgeon if you develop:
- worsening abdominal pain
- sharp or severe pain that lasts several hours
- temperature of 101 degrees or higher
- severe diarrhea
- vomiting
- redness around the incision that is spreading
- increased swelling around the incision
- excessive bruising around the incision
- cloudy fluid coming from the wound
- bright red blood or foul smelling discharge coming from the 
wound
- an increase in drainage from the wound

 
Followup Instructions:
___

----------------------------------------


'''

print(dis_c_df.iloc[idxs[1]].text)
print( '-' * 40)
'''
INDICATION
COMPARISON
TECHNIQUE
FINDINGS
IMPRESSION
'''

print(dis_c_df.iloc[idxs[2]].text)
print( '-' * 40)
'''
EXAMINATION
INDICATION
DOSE
COMPARISON
FINDINGS
Head_CTA
IMPRESSION
'''

 
Name:  ___.           Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: NEUROSURGERY
 
Allergies: 
No Allergies/ADRs on File
 
Attending: ___.
 
Chief Complaint:
Fall
 
Major Surgical or Invasive Procedure:
None
 
History of Present Illness:
___ y/o male on ASA  who was walking with his walker today and
fell backwards striking his head and was found down by his wife.
Patient was found confused but alert when the daughter arrived
ten minutes later. EMS was called and transferred the patient to
___, in route, the patient became cyanotic and
stopped breathing, attempts to intubate the patient failed
secondary to food impaction. Patient was bagged mask ventilated
to ___ . He underwent RSI and had a head CT that revealed a
large holohemespheric right sided SDH with midline shift. 
Patient was transferred to ___ for further care.

 
Past Medical History:
DM 2
HTN
MI x2
CABG
 
Social History:
___
Family History:
NC

'\nEXAMINATION\nINDICATION\nDOSE\nCOMPARISON\nFINDINGS\nHead_CTA\nIMPRESSION\n'